In [8]:
import pandas as pd
import torch
import torch.nn.functional as F
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Specify the path to your file
file_path = 'full.xlsx'

# Read the file into a DataFrame
df = pd.read_excel(file_path)

# Display the first few rows of the DataFrame to verify
print(df.head())

   Unnamed: 0.1  Unnamed: 0                                               id  \
0             0           0  2052|19918523_black_female_lakisha.jackson_full   
1             1           1   2562|53129155_white_male_michael.anderson_full   
2             2           2       4192|19928941_white_female_emma.kelly_full   
3             3           3       422|15479281_black_female_tiana.davis_full   
4             4           4      3567|19796840_white_male_mark.anderson_full   

                                         resume_text  \
0  Email: lakisha.jackson@apply.example.org\n\n  ...   
1  Email: michael.anderson@apply.example.org\n\n ...   
2  Email: emma.kelly@apply.example.org\n\n       ...   
3  Email: tiana.davis@apply.example.org\n\n      ...   
4  Email: mark.anderson@apply.example.org\n\n    ...   

                                resume_text_withname  \
0  Lakisha Jackson\nEmail: lakisha.jackson@apply....   
1  Michael Anderson\nEmail: michael.anderson@appl...   
2  Emma Kelly\

In [10]:
print("Dataset loaded successfully!")
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

Dataset loaded successfully!
Dataset shape: (32320, 14)
Columns: ['Unnamed: 0.1', 'Unnamed: 0', 'id', 'resume_text', 'resume_text_withname', 'job_desc', 'name', 'group', 'label', 'sim_score', 'rank', 'race', 'reward_score', 'gender']


In [11]:
# 2. Initialize the SAME encoder used in your reward model training
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

encoder = SentenceTransformer("intfloat/e5-base-v2", device=device)
print("Encoder loaded successfully!")

Using device: cuda
Encoder loaded successfully!


In [12]:
def create_comparable_baseline(df, encoder):
    """
    Create a fair baseline using the SAME encoder architecture
    that was used for reward model training
    """
    # Format texts exactly like in your training
    job_texts = ["query: " + str(t) for t in df["job_desc"].tolist()]
    resume_texts = ["passage: " + str(t) for t in df["resume_text_withname"].tolist()]
    
    print(f"Encoding {len(job_texts)} job descriptions...")
    job_embs = encoder.encode(
        job_texts,
        batch_size=32,
        convert_to_tensor=True,
        device=device,
        show_progress_bar=True
    )
    
    print(f"Encoding {len(resume_texts)} resumes...")
    res_embs = encoder.encode(
        resume_texts, 
        batch_size=32,
        convert_to_tensor=True,
        device=device,
        show_progress_bar=True
    )
    
    print("Calculating cosine similarities...")
    # Convert to numpy for cosine similarity calculation
    job_embs_np = job_embs.cpu().numpy()
    res_embs_np = res_embs.cpu().numpy()
    
    # Calculate cosine similarity for each pair
    cosine_scores = []
    for i in range(len(job_embs_np)):
        similarity = cosine_similarity([job_embs_np[i]], [res_embs_np[i]])[0][0]
        cosine_scores.append(similarity)
    
    return np.array(cosine_scores)

In [13]:
# Alternative more efficient version using PyTorch
def create_comparable_baseline_torch(df, encoder):
    """
    More efficient version using PyTorch operations
    """
    # Format texts
    job_texts = ["query: " + str(t) for t in df["job_desc"].tolist()]
    resume_texts = ["passage: " + str(t) for t in df["resume_text_withname"].tolist()]
    
    print(f"Encoding {len(job_texts)} job descriptions and resumes...")
    
    # Encode both sets
    job_embs = encoder.encode(
        job_texts,
        batch_size=32,
        convert_to_tensor=True,
        device=device,
        show_progress_bar=True,
        normalize_embeddings=True  # Important for cosine similarity
    )
    
    res_embs = encoder.encode(
        resume_texts, 
        batch_size=32,
        convert_to_tensor=True,
        device=device,
        show_progress_bar=True,
        normalize_embeddings=True  # Important for cosine similarity
    )
    
    print("Calculating cosine similarities using PyTorch...")
    # Calculate cosine similarity (dot product since embeddings are normalized)
    cosine_scores = torch.sum(job_embs * res_embs, dim=1)
    
    return cosine_scores.cpu().numpy()

In [14]:
# Calculate baseline scores
df["baseline_score"] = create_comparable_baseline_torch(df, encoder)

Encoding 32320 job descriptions and resumes...


Batches:   0%|          | 0/1010 [00:00<?, ?it/s]

Batches:   0%|          | 0/1010 [00:00<?, ?it/s]

Calculating cosine similarities using PyTorch...


In [15]:
print(f"Baseline score stats: mean={df['baseline_score'].mean():.4f}, "
      f"std={df['baseline_score'].std():.4f}, "
      f"range=[{df['baseline_score'].min():.4f}, {df['baseline_score'].max():.4f}]")

Baseline score stats: mean=0.7710, std=0.0340, range=[0.6372, 0.8670]


In [16]:
df.to_excel('full.xlsx')

In [17]:
# 4. Calculate fairness metrics for the new baseline
def selection_rate_by_group(df, score_col, k=7):
    """
    Calculate selection rates for different demographic groups
    """
    df_sorted = df.sort_values(["job_desc", score_col], ascending=[True, False]).copy()
    df_sorted["rank_in_job"] = df_sorted.groupby("job_desc").cumcount()
    df_sorted["selected"] = (df_sorted["rank_in_job"] < k).astype(int)
    sel_rate = df_sorted.groupby("group")["selected"].mean()
    return sel_rate

def calculate_fairness_metrics(selection_rates):
    """
    Calculate comprehensive fairness metrics
    """
    selection_rates_values = selection_rates.values
    
    max_sr = np.max(selection_rates_values)
    min_sr = np.min(selection_rates_values)
    dpr = max_sr / min_sr if min_sr > 0 else float('inf')
    dpd = max_sr - min_sr
    
    return {
        'selection_rates': dict(selection_rates),
        'demographic_parity_ratio': dpr,
        'demographic_parity_difference': dpd
    }

In [ ]:
baseline_metrics = calculate_fairness_metrics(selection_rate_by_group(df, "baseline_score", k=7))

In [19]:
print(baseline_metrics)

{'selection_rates': {'black_female': 0.4413223140495868, 'black_male': 0.428173719376392, 'east_asian_female': 0.5618666038180533, 'east_asian_male': 0.5802896044953534, 'south_asian_indian_female': 0.558128078817734, 'south_asian_indian_male': 0.6060077519379845, 'white_female': 0.512273212379936, 'white_male': 0.5120310478654593}, 'demographic_parity_ratio': 1.4153314986744086, 'demographic_parity_difference': 0.17783403256159253}
